## Predição do nível de pobreza doméstica na Costa Rica.

1. Preparar o dataset


In [ ]:
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_train = pd.read_csv("/kaggle/input/costa-rican-household-poverty-prediction/train.csv")
df_test = pd.read_csv("/kaggle/input/costa-rican-household-poverty-prediction/test.csv")

df_train.shape, df_test.shape

### Os dados da competição são fornecidos em dois arquivos: 'train.csv' e 'test.csv'. O conjunto de treinamento possui 9557 linhas e 143 colunas enquanto o conjunto de teste possui 23856 linhas e 142 colunas. O conjunto de treinamento posssui uma coluna a mais, Target, que representa o nível de pobreza em uma escala de 1-4.
### 1 = pobreza extrema
### 2 = pobreza moderada
### 3 = famílias vulneráveis
### 4 = domicílios não vulneráveis

In [ ]:
df_train.head()

In [ ]:
df_test.head()

### Verificar as informações nos dados de treinamento.

In [ ]:
df_train.info()

### Verificar as informações nos dados de teste.

In [ ]:
df_test.info()

### A maioria dos dados é float e int, e poucos casos são object.
### Quero saber quais colunas são do tipo object. Primeiro irei fazer nos dados de test e depois nos dados de train para verificar se há alguma diferença.

In [ ]:
df_test.columns[df_test.dtypes==object]

In [ ]:
df_train.columns[df_train.dtypes==object]

* 'Id' = identificador único.
* 'idhogar' = este é um identificador único para cada família. Ele pode ser usado para criar características domésticas, etc.
* 'dependency' = Taxa de dependência, calculada = (número de membros da família com menos de 19 ou mais de 64 anos)/(número de membros da família entre 19 e 64 anos).
* 'edjefe' = anos de educação masculino.
* 'edjefa' = anos de educação feminino.

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
df_train.select_dtypes('object').head()

### Verificar os valores ausentes nos dados de treinamento.

In [ ]:
df_train.isna().sum()

### Verificar os valores ausentes nos dados de teste.

In [ ]:
df_test.isna().sum()

### Outra maneira de olhar os valores nulos é filtrar por coluna.

In [ ]:
df_train.columns[df_train.isna().sum()!=False]

### No total são 5 colunas que possui valores ausentes.
* 'v2a1' = pagamento mensal do aluguel.
* 'v18q1' = número de tablets que o domiciliar tem.
* 'rez_esc' = anos de atraso escolar.
* 'meaneduc' = educação média para adultos.
* 'SQBmeaned' =  quadrado da média de anos de escolaridade dos adultos (>=18) no domicílio.

In [ ]:
df_test.columns[df_test.isna().sum()!=False]

### Posso olhar em cada coluna separadamente.

In [ ]:
print(df_train['v2a1'].isna().sum())
print(df_train['v18q1'].isna().sum())
print(df_train['rez_esc'].isna().sum())
print(df_train['meaneduc'].isna().sum())
print(df_train['SQBmeaned'].isna().sum())

### Os valores NaN na coluna 'v2a1' são de pessoas que não pagaram o aluguel por algum motivo.

In [ ]:
df_train['v2a1'] = df_train['v2a1'].fillna(0)
df_test['v2a1'] = df_test['v2a1'].fillna(0)

### A coluna 'v18q1' possue uma relação com a coluna 'v18q', essa última diz se alguém da família tem um tablet.

In [ ]:
df_train['v18q1'] = df_train['v18q1'].fillna(0)
df_test['v18q1'] = df_test['v18q1'].fillna(0)

### A coluna 'meaneduc' e 'SQBmeaned' os dois dados.

In [ ]:
df_train[['meaneduc', 'SQBmeaned']].describe()

In [ ]:
df_test[['meaneduc', 'SQBmeaned']].describe()

In [ ]:
df_train['meaneduc'].fillna(0)
df_train['SQBmeaned'].fillna(0)

In [ ]:
df_test['meaneduc'].fillna(0)
df_test['SQBmeaned'].fillna(0)

### Na coluna 'rez_esc'

In [ ]:
df_train['rez_esc'].fillna(0)
df_test['rez_esc'].fillna(0)

### Gráfico de nossa variável target.

In [ ]:
import matplotlib.pyplot as plt
df_train['Target'].plot.hist()

In [ ]:
df_train['Target'].value_counts()

### Existem algumas colunas que são os quadrado de outras colunas como SQBedjefe, SQBmeaned, SQBdepedndency,etc.Talvez elas sejam irrelevantes pois já existe uma coluna normal para elas.

In [ ]:
df_train.head()

In [ ]:
df_train['edjefe'].value_counts()

In [ ]:
df_train['edjefa'].value_counts()

In [ ]:
df_train['dependency'].value_counts()

### Temos valores "yes" e "no" nas colunas 'edjefe','edjefa' e 'dependency', tanto nos dados de treino quanto nos de teste.
### Substituir esses valores por 'yes=1' e 'no=0'.

In [ ]:
mapping = {"yes": 1, "no": 0}

for df in [df_train, df_test]:
    df['dependency'] = df['dependency'].replace(mapping).astype(np.float64)
    df['edjefa'] = df['edjefa'].replace(mapping).astype(np.float64)
    df['edjefe'] = df['edjefe'].replace(mapping).astype(np.float64)

df_train[['dependency', 'edjefa', 'edjefe']].describe()

In [ ]:
df_test.info()

In [ ]:
df_train.drop(['Id'],axis=1)
df_train.drop(['idhogar'],axis=1)

In [ ]:
df_train.select_dtypes('object').head()

In [ ]:
df_train = df_train.drop(columns = ['SQBmeaned', 'agesq'])
df_train.shape

In [ ]:
df_test = df_test.drop(columns = ['SQBmeaned', 'agesq'])
df_test.shape

In [ ]:
df_train.info()

## Agora que terminamos a análise e modificações dos dados podemos seguir em frente para dividir os dados de treinamento.

In [ ]:
from matplotlib import pyplot
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Usando o XBGClassifier

In [ ]:
X=df_train.drop(['Id', 'idhogar', 'Target'], axis=1)

y=df_train['Target']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

X_train.shape, y_train.shape

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

In [ ]:
import lightgbm as lgb

lgb = lgb.LGBMClassifier()
lgb.fit(X_train, y_train)
preds = lgb.predict(X_test)
print(accuracy_score(y_test, preds))
print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))

In [ ]:
z = pd.Series(lgb.predict(X_test),name="Target")
df_entrega = pd.concat([df_test.Id,z], axis=1)
df_entrega.to_csv("/kaggle/working/submission.csv",index=False)